In [ ]:
pip install langchain

In [ ]:
pip install langchain_experimental

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

 ········


In [11]:
pip install -qU langchain-fireworks

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: D:\LangChain\langchain\Scripts\python.exe -m pip install --upgrade pip


In [12]:
os.environ["FIREWORKS_API_KEY"] = getpass.getpass()

 ········


In [13]:
from langchain_fireworks import ChatFireworks
model=ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

messages=[SystemMessage(content="Translate the following from English to Hindi"),
          HumanMessage(content="Hi")]
model.invoke(messages)

AIMessage(content='Hello! In Hindi, "Hi" is translated as "Namaste" or "Hello" (the same as in English). So, you can say "Namaste" or "Hello" to greet someone in Hindi.\n\nNote: "Namaste" is a more traditional and respectful greeting in Hindi, especially when used in a formal or religious context. It is often accompanied by pressing your palms together in front of your chest and slightly bowing your head. On the other hand, "Hello" is a more modern and casual greeting, similar to its usage in English.', response_metadata={'token_usage': {'prompt_tokens': 19, 'total_tokens': 148, 'completion_tokens': 129}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-69054c7b-55c6-49f6-8718-797a95183de2-0')

##### we use a string parser called StrOutputParser to extract only the result string from the AI message

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

In [16]:
result=model.invoke(messages)

parser.invoke(result)

'Hello! In Hindi, "Hi" is translated as "Namaste" or "Hello" (the same as in English). So, you can say "Namaste" or "Hello" to greet someone in Hindi.\n\nNote: "Namaste" is a more traditional and respectful greeting in Hindi, especially when used in a formal or religious context. It is often accompanied by pressing your palms together in front of your chest and slightly bowing your head. On the other hand, "Hello" is a more modern and casual greeting, similar to its usage in English.'

#### the parser can be chained so that it gets called everytime with the chain. this is done by | operator 

In [17]:
chain=model|parser

chain.invoke(messages)

'Hello! In Hindi, "Hi" is translated as "Namaste" or "Hello" (the same as in English). So, you can say "Namaste" or "Hello" to greet someone in Hindi.\n\nNote: "Namaste" is a more traditional and respectful greeting in Hindi, especially when used in a formal or religious context. It is often accompanied by pressing your palms together in front of your chest and slightly bowing your head. On the other hand, "Hello" is a more modern and casual greeting, similar to its usage in English.'

## Prompt Templates

#### they take raw data input from user and return a prompt ready to be passed into the model.

In [18]:
from langchain_core.prompts import ChatPromptTemplate

In [20]:
system_template="Translate the following into {language}:"

In [21]:
prompt_template=ChatPromptTemplate.from_messages([("system",system_template),("user","{text}")])

In [22]:
result=prompt_template.invoke({"language":"Hindi","text":"Hi"})

In [23]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Hindi:'), HumanMessage(content='Hi')])

In [24]:
#to access msg directly
result.to_messages()

[SystemMessage(content='Translate the following into Hindi:'),
 HumanMessage(content='Hi')]

#### chaining everything

In [25]:
chain=prompt_template|model|parser

chain.invoke({"language":"hindi","text":"Hi"})

'Hello! In Hindi, "Hi" is translated as "Namaste" or "Hello" (informally).\n\nNote: "Namaste" is a traditional Indian greeting, while "Hello" is used in both formal and informal settings, similar to English.'

# Serving with LangServe

In [ ]:
pip install "langserve[all]"

In [29]:
## create a server file

In [34]:
## run the file on cmd

# Client

In [35]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "hindi", "text": "hi"})

'The word "hi" is a common greeting in English, similar to saying "hello" in English or "namaste" in Hindi. In Hindi, you can say "hi" as "हाई" (pronounced: hai). However, it\'s worth noting that "hi" is not a commonly used greeting in Hindi, and you\'re more likely to hear "namaste" or "pranaam" (a more formal greeting) in Hindi-speaking regions.'